<a href="https://colab.research.google.com/github/Kasuletrevor/Kasuletrevor/blob/main/FSC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [90]:
import os
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import librosa.display
import librosa
import IPython.display as ipd
# from IPython.display import Audio
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [39]:
os.getcwd()

'/content'

In [ ]:
# from google.colab import files

# # Open the file and read its contents
# with files.open('filename.txt', 'r') as f:
#   contents = f.read()

In [ ]:
# !pip install IPython

# Kaggle API

In [4]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kasulejohntrevor","key":"2cd289afcd50553c35b2a4aedbc160f3"}'}

In [7]:
#Make a directory named kaggle and copy the kaggle.json file there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# change the permission of the file
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
# !your dataset api command
!kaggle datasets download -d tommyngx/fluent-speech-corpus 

100% 1.44G/1.44G [00:13<00:00, 110MB/s]
100% 1.44G/1.44G [00:13<00:00, 114MB/s]


In [9]:
from zipfile import ZipFile
file_name = 'fluent-speech-corpus.zip' #the file is your dataset exact name
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('Done')

Done


# CHECKING DATASET AND PATHS

In [100]:
os.getcwd()

'/content'

In [101]:
ls

drive/                           fluent-speech-corpus.zip  sample_data/
fluent_speech_commands_dataset/  kaggle.json


In [102]:
ls fluent_speech_commands_dataset

 data/  'Fluent Speech Commands Public License.pdf'   readme.md   wavs/


In [103]:
ls fluent_speech_commands_dataset/data/

speaker_demographics.csv  test_data.csv  train_data.csv  valid_data.csv


In [104]:
df = pd.read_csv("fluent_speech_commands_dataset/data/train_data.csv")

In [105]:
df.head()

,Unnamed: 0,path,speakerId,transcription,action,object,location
0,0,wavs/speakers/2BqVo8kVB2Skwgyb/0a3129c0-4474-1...,2BqVo8kVB2Skwgyb,Change language,change language,none,none
1,1,wavs/speakers/2BqVo8kVB2Skwgyb/0ee42a80-4474-1...,2BqVo8kVB2Skwgyb,Resume,activate,music,none
2,2,wavs/speakers/2BqVo8kVB2Skwgyb/144d5be0-4474-1...,2BqVo8kVB2Skwgyb,Turn the lights on,activate,lights,none
3,3,wavs/speakers/2BqVo8kVB2Skwgyb/1811b6e0-4474-1...,2BqVo8kVB2Skwgyb,Switch on the lights,activate,lights,none
4,4,wavs/speakers/2BqVo8kVB2Skwgyb/1d9f3920-4474-1...,2BqVo8kVB2Skwgyb,Switch off the lights,deactivate,lights,none


In [106]:
df['transcription'].nunique()

248

In [107]:
df["action"]

0        change language
1               activate
2               activate
3               activate
4             deactivate
              ...       
23127    change language
23128    change language
23129    change language
23130    change language
23131         deactivate
Name: action, Length: 23132, dtype: object

In [108]:
filename

'fluent_speech_commands_dataset/wavs/speakers/ObdQbr9wyDfbmW4E/9df442d0-443f-11e9-beaa-1f3e1eab7bff.wav'

In [109]:
df_actions = df['action'].unique()
actions= {}
for i, action in enumerate(df_actions):
    actions[i] = action

print(actions)

{0: 'change language', 1: 'activate', 2: 'deactivate', 3: 'increase', 4: 'decrease', 5: 'bring'}


In [110]:
df_objects = df['object'].unique()
objects= {}
for i, object in enumerate(df_objects):
    objects[i] = object
    
print(objects)

{0: 'none', 1: 'music', 2: 'lights', 3: 'volume', 4: 'heat', 5: 'lamp', 6: 'newspaper', 7: 'juice', 8: 'socks', 9: 'Chinese', 10: 'Korean', 11: 'English', 12: 'German', 13: 'shoes'}


In [111]:
train_df = df.copy()

In [112]:
train_df.head()

,Unnamed: 0,path,speakerId,transcription,action,object,location
0,0,wavs/speakers/2BqVo8kVB2Skwgyb/0a3129c0-4474-1...,2BqVo8kVB2Skwgyb,Change language,change language,none,none
1,1,wavs/speakers/2BqVo8kVB2Skwgyb/0ee42a80-4474-1...,2BqVo8kVB2Skwgyb,Resume,activate,music,none
2,2,wavs/speakers/2BqVo8kVB2Skwgyb/144d5be0-4474-1...,2BqVo8kVB2Skwgyb,Turn the lights on,activate,lights,none
3,3,wavs/speakers/2BqVo8kVB2Skwgyb/1811b6e0-4474-1...,2BqVo8kVB2Skwgyb,Switch on the lights,activate,lights,none
4,4,wavs/speakers/2BqVo8kVB2Skwgyb/1d9f3920-4474-1...,2BqVo8kVB2Skwgyb,Switch off the lights,deactivate,lights,none


In [113]:
# uniq = df[['action', 'object']].apply(lambda x: x[0] if x[0] == x[1] else np.NaN, axis=1)

# print(uniq)

In [114]:
# uniq = uniq.dropna()
# print(uniq)

In [115]:
# uniq = list(set(df['action']).intersection(set(df['object'])))
# print(uniq)


In [116]:
# import pandas as pd

# # Create a sample data frame
# df = pd.DataFrame({'col1': ['a', 'b', 'c'], 'col2': ['d', 'e', 'f']})

# # Concatenate the values in col1 and col2 and store the result in a new column
# df['new_col'] = df['col1'] + df['col2']

# print(df)

In [117]:
def concat(row):
  return row['action'] + ' ' + row['object'] + ' ' +row['location']

df['intent'] = df.apply(concat, axis=1)
# print(df)

In [118]:
df.tail()

,Unnamed: 0,path,speakerId,transcription,action,object,location,intent
23127,23127,wavs/speakers/zZezMeg5XvcbRdg3/b946b340-45e0-1...,zZezMeg5XvcbRdg3,I need to practice my Chinese. Switch the lang...,change language,Chinese,none,change language Chinese none
23128,23128,wavs/speakers/zZezMeg5XvcbRdg3/beb27cb0-45e0-1...,zZezMeg5XvcbRdg3,I need to practice my German. Switch the language,change language,German,none,change language German none
23129,23129,wavs/speakers/zZezMeg5XvcbRdg3/c45f94e0-45e0-1...,zZezMeg5XvcbRdg3,I need to practice my Korean. Switch the language,change language,Korean,none,change language Korean none
23130,23130,wavs/speakers/zZezMeg5XvcbRdg3/ca60c080-45e0-1...,zZezMeg5XvcbRdg3,I need to practice my English. Switch the lang...,change language,English,none,change language English none
23131,23131,wavs/speakers/zZezMeg5XvcbRdg3/cf10b400-45e0-1...,zZezMeg5XvcbRdg3,Turn the washroom lights off,deactivate,lights,washroom,deactivate lights washroom


In [119]:
df['intent'].nunique()

31

In [120]:
import random

# Generate a random integer between 1 and 3793
f = random.randint(1, 3793)
print(f)


605


In [121]:
filepath = df['path'][f]

In [122]:
filepath = df['path'][f]
filepath = filepath.split('/')
filename = 'fluent_speech_commands_dataset'
for i in filepath:
    filename = os.path.join(filename,i)
data,sr = librosa.load(filename)

In [123]:
filepath = filepath.split('/')
filepath

AttributeError: ignored

In [ ]:
filename = 'fluent_speech_commands_dataset'
for i in filepath:
    filename = os.path.join(filename,i)
filename

In [ ]:
data,sr = librosa.load(filename)

In [ ]:
sr

In [ ]:
data

In [ ]:
plt.plot(data)
plt.show()

In [ ]:
ipd.Audio(filename)

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(data, sr=sr)

In [ ]:
X = librosa.stft(data)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')

# ***UNUSED CODE***

> This section contains code that I will need for reference.




In [ ]:
# # Load the audio data and sample rate
# data, sr = librosa.load('audio_file.wav')

# # Pre-process the audio data (optional)
# data = librosa.feature.rmse(data)

# # Compute the power spectrum
# power_spectrum = np.abs(librosa.stft(data)) ** 2

# # Extract the MFCCs
# mfccs = librosa.feature.mfcc(S=power_spectrum, n_mfcc=20)

# # Perform CMN on the MFCCs (optional)
# mfccs = librosa.util.normalize(mfccs, axis=1)

# This code will load the audio data from the file audio_file.wav, apply root mean square (RMS) energy normalization to the data, compute the power spectrum using the short-time Fourier transform (STFT), extract the MFCCs using a Mel filterbank with 20 filters, and perform CMN on the MFCCs. 
# The resulting MFCCs will be a 2D array, with one row for each frame of audio and one column for each MFCC coefficient.

In [ ]:
import wave
import struct

# Open the audio file as a wave object
# audio_file = wave.open("example.wav", "rb")
audio_file = wave.open(filename, "rb")

# Extract the sampling rate and number of channels from the wave object
sampling_rate = audio_file.getframerate()
num_channels = audio_file.getnchannels()

# Read the entire audio signal as a sequence of frames
audio_signal = audio_file.readframes(-1)

# Convert the frames to a sequence of samples, using the specified
# number of channels and the specified sample width
num_samples = len(audio_signal) // (num_channels * audio_file.getsampwidth())
samples = struct.unpack("{}h".format(num_samples * num_channels), audio_signal)

In [ ]:
import librosa

# Load the audio signal from a file
# audio_signal, sampling_rate = librosa.load("example.wav")
audio_signal, sampling_rate = librosa.load(filename)

# Convert the audio signal to a sequence of samples
samples = librosa.util.buf_to_float(audio_signal, n_bytes=2, dtype=np.float32)

In [ ]:
samples.shape

(63976,)

In [ ]:
# import numpy as np

# # Reshape the MFCCs data into a 3D array
# timesteps = mfccs.shape[0]
# num_mfccs = mfccs.shape[1]
# mfccs_3d = np.expand_dims(mfccs, axis=2)

# # Create the LSTM model
# model = keras.Sequential()
# model.add(keras.layers.LSTM(32, input_shape=(timesteps, num_mfccs)))
# model.add(keras.layers.Dense(1, activation='sigmoid'))

# # Compile and fit the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(mfccs_3d, labels, epochs=10)


In [ ]:
# import json
# import os
# import math
# import librosa

# DATASET_PATH = "path/to/marsyas/dataset"
# JSON_PATH = "data_10.json"
# SAMPLE_RATE = 22050
# TRACK_DURATION = 30 # measured in seconds
# SAMPLES_PER_TRACK = SAMPLE_RATE * TRACK_DURATION


# def save_mfcc(dataset_path, json_path, num_mfcc=13, n_fft=2048, hop_length=512, num_segments=5):
#     """Extracts MFCCs from music dataset and saves them into a json file along witgh genre labels.

#         :param dataset_path (str): Path to dataset
#         :param json_path (str): Path to json file used to save MFCCs
#         :param num_mfcc (int): Number of coefficients to extract
#         :param n_fft (int): Interval we consider to apply FFT. Measured in # of samples
#         :param hop_length (int): Sliding window for FFT. Measured in # of samples
#         :param: num_segments (int): Number of segments we want to divide sample tracks into
#         :return:
#         """

#     # dictionary to store mapping, labels, and MFCCs
#     data = {
#         "mapping": [],
#         "labels": [],
#         "mfcc": []
#     }

#     samples_per_segment = int(SAMPLES_PER_TRACK / num_segments)
#     num_mfcc_vectors_per_segment = math.ceil(samples_per_segment / hop_length)

#     # loop through all genre sub-folder
#     for i, (dirpath, dirnames, filenames) in enumerate(os.walk(dataset_path)):

#         # ensure we're processing a genre sub-folder level
#         if dirpath is not dataset_path:

#             # save genre label (i.e., sub-folder name) in the mapping
#             semantic_label = dirpath.split("/")[-1]
#             data["mapping"].append(semantic_label)
#             print("\nProcessing: {}".format(semantic_label))

#             # process all audio files in genre sub-dir
#             for f in filenames:

# 		# load audio file
#                 file_path = os.path.join(dirpath, f)
#                 signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

#                 # process all segments of audio file
#                 for d in range(num_segments):

#                     # calculate start and finish sample for current segment
#                     start = samples_per_segment * d
#                     finish = start + samples_per_segment

#                     # extract mfcc
#                     mfcc = librosa.feature.mfcc(signal[start:finish], sample_rate, n_mfcc=num_mfcc, n_fft=n_fft, hop_length=hop_length)
#                     mfcc = mfcc.T

#                     # store only mfcc feature with expected number of vectors
#                     if len(mfcc) == num_mfcc_vectors_per_segment:
#                         data["mfcc"].append(mfcc.tolist())
#                         data["labels"].append(i-1)
#                         print("{}, segment:{}".format(file_path, d+1))

#     # save MFCCs to json file
#     with open(json_path, "w") as fp:
#         json.dump(data, fp, indent=4)
        
        
# if __name__ == "__main__":
#     save_mfcc(DATASET_PATH, JSON_PATH, num_segments=10)

In [ ]:
# from scipy.io.wavfile import read
# from python_speech_features import mfcc

# mfccs = []
# labels = []

# for index, row in df.iterrows():
#     # Load the audio file
#     fs, signal = read(row['path'])
    
#     # Extract the MFCCs
#     mfcc_features = mfcc(signal, fs)
    
#     # Append the MFCCs and label to the list
#     mfccs.append(mfcc_features)
#     labels.append(row['label'])


In [59]:
# import librosa

# mfccs = []
# labels = []

# for index, row in df.iterrows():
#     # Load the audio file
#     signal, fs = librosa.load(row['path'])
    
#     # Extract the MFCCs
#     mfcc_features = librosa.feature.mfcc(signal, fs, n_mfcc=40)
    
#     # Append the MFCCs and label to the list
#     mfccs.append(mfcc_features)
#     labels.append(row['label'])


# **Data** **Preprocessing**

## 1. Loading training set

In [139]:
df = pd.read_csv("fluent_speech_commands_dataset/data/train_data.csv")

In [94]:
def concat(row):
  return row['action'] + ' ' + row['object'] + ' ' +row['location']

In [95]:
df.head()

,Unnamed: 0,path,speakerId,transcription,action,object,location
0,0,wavs/speakers/2BqVo8kVB2Skwgyb/0a3129c0-4474-1...,2BqVo8kVB2Skwgyb,Change language,change language,none,none
1,1,wavs/speakers/2BqVo8kVB2Skwgyb/0ee42a80-4474-1...,2BqVo8kVB2Skwgyb,Resume,activate,music,none
2,2,wavs/speakers/2BqVo8kVB2Skwgyb/144d5be0-4474-1...,2BqVo8kVB2Skwgyb,Turn the lights on,activate,lights,none
3,3,wavs/speakers/2BqVo8kVB2Skwgyb/1811b6e0-4474-1...,2BqVo8kVB2Skwgyb,Switch on the lights,activate,lights,none
4,4,wavs/speakers/2BqVo8kVB2Skwgyb/1d9f3920-4474-1...,2BqVo8kVB2Skwgyb,Switch off the lights,deactivate,lights,none


In [133]:
df['intent'] = df.apply(concat, axis=1)

In [97]:
mfccs = []
labels = []

In [ ]:
#function to load data
def load_set(path):
  #load dataframe
  df = pd.read_csv("path")
  df['intent'] = df.apply(concat, axis=1)
  mfccs = []
  labels = []
  for index, row in df.iterrows():
    filepath = row['path']
    filepath = filepath.split('/')
    filename = 'fluent_speech_commands_dataset'
    for i in filepath:
      filename = os.path.join(filename,i)
    
    #filename extracted
    # Load the audio file
    data,sr = librosa.load(filename)

    #extract mfccs
    mfcc_features = librosa.feature.mfcc(data, sr, n_mfcc=40)

    mfccs.append(mfcc_features)
    labels.append(row['intent'])
    # Determine the maximum number of MFCCs
    max_mfccs = max([mfcc.shape[1] for mfcc in mfccs])

    # Pad the MFCC arrays with zeros so that they all have the same shape
    padded_mfccs = [np.pad(mfcc, ((0,0), (0, max_mfccs - mfcc.shape[1])), 'constant') for mfcc in mfccs]

    # Stack the padded MFCC arrays into a single matrix
    mfccs = np.vstack(padded_mfccs)


    # Convert the labels to a numerical format using LabelEncoder

    le = LabelEncoder()
    labels = le.fit_transform(labels)

    return mfccs, labels

In [98]:
for index, row in df.iterrows():
    filepath = row['path']
    filepath = filepath.split('/')
    filename = 'fluent_speech_commands_dataset'
    for i in filepath:
      filename = os.path.join(filename,i)
    
    #filename extracted
    # Load the audio file
    data,sr = librosa.load(filename)

    #extract mfccs
    mfcc_features = librosa.feature.mfcc(data, sr, n_mfcc=40)

    mfccs.append(mfcc_features)
    labels.append(row['intent'])
    

KeyboardInterrupt: ignored

In [ ]:
# #  it means that the MFCC arrays have different shapes. 
# This can happen if the audio files have different lengths, or if you are using a variable number of MFCCs for different audio files.

# # To fix this issue, you can pad the MFCC arrays with zeros so that they all have the same shape, like this:

In [65]:
# Determine the maximum number of MFCCs
max_mfccs = max([mfcc.shape[1] for mfcc in mfccs])

# Pad the MFCC arrays with zeros so that they all have the same shape
padded_mfccs = [np.pad(mfcc, ((0,0), (0, max_mfccs - mfcc.shape[1])), 'constant') for mfcc in mfccs]

# Stack the padded MFCC arrays into a single matrix
mfccs = np.vstack(padded_mfccs)


# Convert the labels to a numerical format using LabelEncoder

le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(mfccs, labels, test_size=0.2, random_state=42)


In [66]:
X_train = mfccs
y_train = labels

In [91]:
X_train.shape

(925280, 570)

In [88]:
y_train.shape

(23132,)

## 2. Loading test set

In [67]:
df1 = pd.read_csv("fluent_speech_commands_dataset/data/test_data.csv")

In [ ]:
df1['intent'] = df1.apply(concat, axis=1)

In [ ]:
mfccs = []
labels = []

In [73]:
for index, row in df1.iterrows():
    filepath = row['path']
    filepath = filepath.split('/')
    filename = 'fluent_speech_commands_dataset'
    for i in filepath:
      filename = os.path.join(filename,i)
    
    #filename extracted
    # Load the audio file
    data,sr = librosa.load(filename)

    #extract mfccs
    mfcc_features = librosa.feature.mfcc(data, sr, n_mfcc=40)

    mfccs.append(mfcc_features)
    labels.append(row['intent'])

In [74]:
max_mfccs = max([mfcc.shape[1] for mfcc in mfccs])

# Pad the MFCC arrays with zeros so that they all have the same shape
padded_mfccs = [np.pad(mfcc, ((0,0), (0, max_mfccs - mfcc.shape[1])), 'constant') for mfcc in mfccs]

# Stack the padded MFCC arrays into a single matrix
mfccs = np.vstack(padded_mfccs)


# Convert the labels to a numerical format
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [75]:
X_test = mfccs
y_test = labels

In [80]:
np.max(labels)

30

# **Model** development

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation

num_classes = 30
model = Sequential()
model.add(Dense(256, input_dim=mfccs.shape[1]))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [82]:
model.fit(X_train, y_train, epochs=10, batch_size=32)

score = model.evaluate(X_test, y_test, batch_size=32)

ValueError: ignored

# sincnet

In [134]:
filepath = df['path'][f]
filepath = filepath.split('/')
filename = 'fluent_speech_commands_dataset'
for i in filepath:
    filename = os.path.join(filename,i)
data,sr = librosa.load(filename)

In [153]:
import librosa

# Load the audio signal from a file
# audio_signal, sampling_rate = librosa.load("example.wav")
audio_signal, sampling_rate = librosa.load(filename)

# Convert the audio signal to a sequence of samples
sampled = librosa.util.buf_to_float(audio_signal, n_bytes=2, dtype=np.float32)

In [154]:
sampled.dtype

dtype('float32')

In [155]:
sampled.shape

(90318,)

In [ ]:
!pip install sincnet-tensorflow

In [145]:
#function to load data
def load_set(path):
  #load dataframe
  df = pd.read_csv(path)
  df['intent'] = df.apply(concat, axis=1)
  samples = []
  labels = []
  for index, row in df.iterrows():
    filepath = row['path']
    filepath = filepath.split('/')
    filename = 'fluent_speech_commands_dataset'
    for i in filepath:
      filename = os.path.join(filename,i)
    
    #filename extracted
    # Load the audio file
    data,sr = librosa.load(filename)

    # Convert the audio signal to a sequence of samples
    sample = librosa.util.buf_to_float(audio_signal, n_bytes=2, dtype=np.float32)
    

    samples.append(sample)
    labels.append(row['intent'])

  return samples, labels

In [146]:
Xtrain, ytrain = load_set("fluent_speech_commands_dataset/data/train_data.csv")


In [156]:
Xtrain[0].shape

(90318,)

In [151]:
ytrain = np.array(ytrain)

In [152]:
ytrain.shape

(23132,)

In [137]:
    # Convert the labels to a numerical format using LabelEncoder

    le = LabelEncoder()
    labels = le.fit_transform(labels)

In [132]:
from tensorflow.keras.layers import Dense, Conv1D
from tensorflow.keras.layers import LeakyReLU, BatchNormalization, Flatten, MaxPooling1D, Input

from sincnet_tensorflow import SincConv1D, LayerNorm


out_dim = 50 #number of classes

sinc_layer = SincConv1D(N_filt=64,
                        Filt_dim=129,
                        fs=16000,
                        stride=16,
                        padding="SAME")


inputs = Input((32000, 1))

x = sinc_layer(inputs)
x = LayerNorm()(x)

x = LeakyReLU(alpha=0.2)(x)
x = MaxPooling1D(pool_size=2)(x)


x = Conv1D(64, 3, strides=1, padding='valid')(x)
x = BatchNormalization(momentum=0.05)(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPooling1D(pool_size=2)(x)

x = Conv1D(64, 3, strides=1, padding='valid')(x)
x = BatchNormalization(momentum=0.05)(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPooling1D(pool_size=2)(x)

x = Conv1D(128, 3, strides=1, padding='valid')(x)
x = BatchNormalization(momentum=0.05)(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPooling1D(pool_size=2)(x)

x = Conv1D(128, 3, strides=1, padding='valid')(x)
x = BatchNormalization(momentum=0.05)(x)
x = LeakyReLU(alpha=0.2)(x)
x = MaxPooling1D(pool_size=2)(x)

x = Flatten()(x)

x = Dense(256)(x)
x = BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
x = LeakyReLU(alpha=0.2)(x)

x = Dense(256)(x)
x = BatchNormalization(momentum=0.05, epsilon=1e-5)(x)
x = LeakyReLU(alpha=0.2)(x)

prediction = Dense(out_dim, activation='softmax')(x)
model = tf.keras.models.Model(inputs=inputs, outputs=prediction)

model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 32000, 1)]        0         
                                                                 
 sinc_conv1d (SincConv1D)    (None, 2000, 64)          128       
                                                                 
 layer_norm (LayerNorm)      (None, 2000, 64)          128       
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 2000, 64)          0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1000, 64)         0         
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 998, 64)           12352     
                                                             

In [127]:
# Load the raw audio signal and extract the frames
signal, sr = librosa.load(filename)
frames = librosa.util.frame(signal, frame_length=200, hop_length=10).T

# Calculate the number of frames and features
n_frames, n_features = frames.shape

frames.shape

(4496, 200)